In [ ]:
%%time
from google.colab import drive
drive.mount("/content/drive")
!unzip "/content/drive/MyDrive/Colab Notebooks/AMEX-data/amex-data.zip"

Mounted at /content/drive
Archive:  /content/drive/MyDrive/Colab Notebooks/AMEX-data/amex-data.zip
  inflating: test.csv                
  inflating: train.csv               
  inflating: train_labels.csv        
CPU times: user 1.89 s, sys: 338 ms, total: 2.23 s
Wall time: 3min 39s


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
!pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as
import gc
from colorama import Style, Fore
import sys
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
cat_features = ['B_30_0.0', 'B_30_1.0', 'B_30_2.0','B_38_1.0','B_38_2.0','B_38_3.0','B_38_4.0',
            'B_38_5.0','B_38_6.0','B_38_7.0','D_114_0.0','D_114_1.0','D_116_0.0','D_116_1.0',
            'D_117_-1.0','D_117_1.0','D_117_2.0','D_117_3.0','D_117_4.0','D_117_5.0','D_117_6.0',
            'D_120_0.0','D_120_1.0','D_126_-1.0','D_126_0.0','D_126_1.0','D_63_CL','D_63_CO','D_63_CR',
            'D_63_XL','D_63_XM','D_63_XZ','D_64_O','D_64_R','D_64_U','D_68_1.0',
            'D_68_2.0','D_68_3.0','D_68_4.0','D_68_5.0','D_68_6.0','R_2_0.0','R_2_1.0','B_8_0.0',
            'B_8_1.0','S_6_0.0','S_6_1.0','D_54_0.0','D_54_1.0','R_4_0.0','R_4_1.0','P_4_0.0',
            'P_4_1.0','B_33_0.0','B_33_1.0','D_103_0.0','D_103_1.0','D_104_0.0','D_104_1.0',
            'R_27_0.0','R_27_1.0','D_112_0.0','D_112_1.0','D_123_0.0','D_123_1.0','D_127_0.0',
            'D_127_1.0','D_128_0.0','D_128_1.0','D_129_0.0','D_129_1.0','D_130_0.0','D_130_1.0',
            'D_131_0.0','D_131_1.0','D_139_0.0','D_139_1.0','D_141_0.0','D_141_0.9','D_143_0.0',
            'D_143_1.0']

In [ ]:
NUM_FOLDS=5

In [ ]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [ ]:
df = pd.read_csv('train.csv', encoding = 'utf-8', sep = ';')
labels = pd.read_csv("train_labels.csv")
train_answers = pd.read_csv("train_answers.csv")
df = df.merge(labels, on='customer_ID')
sys.getsizeof(df)/2**20

2959.9750328063965

In [ ]:
df = pd.concat([df, train_answers], axis=1)

In [ ]:
X = df.drop(['customer_ID', 'target', 'D_64_-1', 'D_68_0.0'], axis = 1)
y = df['target']
del df
_ = gc.collect()

In [ ]:
cbc = CatBoostClassifier(
                         learning_rate = 0.05, 
                         n_estimators = 2000,
                         max_depth = 5, 
                         random_state=42, 
                         max_leaves = 50,
                         grow_policy = 'Lossguide',
                         task_type="GPU",
                         verbose = 250
)

xgb = XGBClassifier(objective='binary:logistic', 
                    n_estimators = 1000, 
                    random_state=23, 
                    learning_rate = 0.3, 
                    max_depth=6, 
                    tree_method='gpu_hist', 
                    verbosity = 3)

models = [xgb, cbc]

In [ ]:
test_answers = pd.read_csv("test_answers.csv")

In [ ]:
skf = StratifiedKFold(n_splits=NUM_FOLDS)
train_answers, test_answers = [], []
for i, model in enumerate(models):
    print(Style.BRIGHT+"{0}th model fitting. Model: {1}".format(i+1 ,model.__str__()) + Style.RESET_ALL)
    answers = np.array([])
    mean_test_answers = []
    for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        print(Fore.BLUE + "#" * 10, f"Fold {fold+1}", "#" * 10 + Style.RESET_ALL)
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]
        X_test = X.iloc[test_index]
        y_test = y.iloc[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]
        score = amex_metric(
            pd.DataFrame({"target": y_test}).reset_index(drop=True), 
            pd.DataFrame({"prediction": y_pred}).reset_index(drop=True),
        )
        print("amex-metric: {0}".format(score))
        answers = np.hstack((answers, y_pred))
        print('predict for test on current fold...')
        test_chunks = pd.read_csv('test.csv', chunksize=20000, encoding = 'utf-8', sep = ';')
        answers_fold = np.array([])
        ids = []
        START = 0
        SIZE = 20000
        for test in test_chunks:
            ids.append(test['customer_ID'])
            X_submit = test.drop(['customer_ID'], axis=1)
            X_submit = pd.concat([X_submit, test_answers.iloc[START:SIZE]], axis=1)
            SIZE +=20000
            START += 20000
            del test
            _ = gc.collect()
            y_pred_submit = model.predict_proba(X_submit)[:, 1]
            del X_submit
            _ = gc.collect()
            answers_fold = np.hstack((answers_fold, y_pred_submit))
        mean_test_answers.append(answers_fold)

    test_answers.append(np.array(mean_test_answers).mean(axis=0))
    train_answers.append(answers)
    print(Fore.BLUE + "#" * 28, "\n" + Style.RESET_ALL)
train_answers = np.array(train_answers)
test_answers = np.array(test_answers)

1th model fitting. Model: XGBClassifier(learning_rate=0.3, max_depth=6, n_estimators=1000,
              random_state=23, verbosity=3)
########## Fold 1 ##########
[17:31:23] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[17:32:10] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[17:32:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[17:33:44] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[17:34:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[17:35:17] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[17:36:03] INFO: /workspace/src/tree/updater_prune.cc:74

KeyboardInterrupt: ignored

[20:15:51] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=6
[20:16:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[20:17:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=6
[20:18:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=6
[20:18:51] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[20:19:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[20:20:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[20:21:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra

In [ ]:
series = pd.Series()
for ser in ids:
    series = series.append(ser)

In [ ]:
subm = pd.DataFrame({'customer_ID': series, 'prediction': test_answers.mean(axis=0)})

In [ ]:
subm = subm.drop_duplicates(subset=['customer_ID'])

In [ ]:
subm.to_csv('submit.csv',index=False)